### モデルフリー強化学習備忘録
問題形式でモデルフリー強化学習における備忘録を記します。なお、ページ内の表記は青本に従います。

#### 価値関数の推定
１．履歴データから価値関数を推定する方法1つ説明してください。

モンテカルロ推定という方法があります。ある状態に対して、その状態に訪れた時からの総報酬の和を、履歴データの中でその状態に訪れた回数で割る方法です。
終端のタイムステップより少し前のタイムステップまでの値で推定します
$$\hat{V}(s):=\frac{\sum_{t=0}^{T^{\prime}} \mathbb{I}_{\left\{s=s_t\right\}} c_t}{\sum_{t=0}^{T^{\prime}} \mathbb{I}_{\left\{s=s_t\right\}}}, \quad \forall s \in\left\{s \in \mathcal{S}: \sum_{t=0}^{T^{\prime}} \mathbb{I}_{\left\{s=s_t\right\}}>0\right\}$$
２．履歴データからベルマン作用素を近似してVI,PIを解きたいというモチベーションがあります。どのようにベルマン作用素を近似するか説明してください。

以下のような作用素を定義して近似します。
$$
\begin{aligned}
& \hat{\mathrm{B}}\left(v ; h_T^\pi\right)(s) \triangleq \begin{cases}\frac{1}{\sum_{t=0}^{T-1} \mathbb{I}_{\left\{s_t=s\right\}}} \sum_{t=0}^{T-1} \mathbb{I}_{\left\{s_t=s\right\}}\left(r_t+\gamma v\left(s_{t+1}\right)\right) & \left(\sum_{t=0}^{T-1} \mathbb{I}_{\left\{s_t=s\right\}}>0\right) \\
v(s) & (\text { それ以外 })\end{cases}
\end{aligned}
$$
３．既にある履歴データから学習する方法と、データを逐次的に追加して学習する方法があります。前者を説明してください。

2の式を使って以下のように価値関数を更新します。ベルマン作用素は縮小性があるので不動点に単調収束します。
$$
\hat{V}(s):=\hat{\mathrm{B}}\left(\hat{V} ; h_T^\pi\right)(s), \quad \forall s \in \mathcal{S}
$$
不動点は以下のようになりますが、履歴データがすべて存在しているので繰り返し更新ではなく解析的に不動点となる価値関数を求めることができます。
$$
\hat{V}_{\infty}(s)=\hat{\mathrm{B}}\left(\hat{V}_{\infty} ; h_T^\pi\right)(s), \quad \forall s \in \mathcal{S}
$$
４．既にある履歴データから学習する方法と、データを逐次的に追加して学習する方法があります。後者を説明してください。収束性なども説明してください。

データが逐次的に追加されていくなかで価値関数を更新する方法があります。TD法を導入します。
$$
\hat{V}\left(s_t\right):=\left(1-\alpha_t\right) \hat{V}\left(s_t\right)+\alpha_t \hat{\mathrm{~B}}\left(\hat{V} ;\left\{s_t, r_t, s_{t+1}\right\}\right)\left(s_t\right)
$$
収束性のための条件はロビンスモンロー条件です。
$$
\alpha_t \geq 0 \quad\left(\forall t \in \mathbb{N}_0\right), \quad \sum_{t=0}^{\infty} \alpha_t=\infty, \quad \sum_{t=0}^{\infty} \alpha_t^2<\infty
$$
##### Q学習
５．Q学習について説明してください。

行動価値関数に関するベルマン最適作用素と近似ベルマン最適作用素を定義します。
$$
\begin{aligned}
& \Upsilon_* q(s, a) \triangleq \mathbb{E}\left\{g(s, a)+\gamma \max _{a^{\prime} \in \mathcal{A}} q\left(S_{t+1}, a^{\prime}\right) \mid S_t=s, A_t=a\right\}, \quad \forall(s, a) \in \mathcal{S} \times \mathcal{A}
\end{aligned}
$$
$$
\begin{aligned}
& \hat{\Upsilon}_*\left(q ; h_T\right)(s, a) \\
& \triangleq \begin{cases}\frac{\sum_{t=0}^{T-1} \mathbb{I}_{\left\{s_t=s\right\}} \mathbb{I}_{\left\{a_t=a\right\}}\left(r_t+\gamma \max _{a^{\prime} \in \mathcal{A}} q\left(s_{t+1}, a^{\prime}\right)\right)}{\sum_{t=0}^{T-1} \mathbb{I}_{\left\{s_t=s\right\}} \mathbb{I}_{\left\{a_t=a\right\}}} & \left(\sum_{t=0}^{T-1} \mathbb{I}_{\left\{s_t=s\right\}} \mathbb{I}_{\left\{a_t=a\right\}}>0\right) \\
q(s, a) & \text { (それ以外) }, \quad \forall(s, a) \in \mathcal{S} \times \mathcal{A}\end{cases}
\end{aligned}
$$
Q学習はTD法を活用して以下のように定義されます。なお、**最適**作用素を用いているのでオフポリシーです。
$$
\hat{Q}\left(s_t, a_t\right):=\left(1-\alpha_t\right) \hat{Q}\left(s_t, a_t\right)+\alpha_t \hat{\Upsilon}_*\left(\hat{Q} ;\left\{s_t, a_t, r_t, s_{t+1}\right\}\right)\left(s_t, a_t\right)
$$
形を少し変えると以下のようになり、TD誤差も一緒に定義されます。
$$
\hat{Q}\left(s_t, a_t\right):=\hat{Q}\left(s_t, a_t\right)+\alpha_t \delta_t^{(\mathrm{q})}
$$
$$
\delta_t^{(\mathrm{q})} \triangleq r_t+\gamma \max _{a^{\prime} \in \mathcal{A}} \hat{Q}\left(s_{t+1}, a^{\prime}\right)-\hat{Q}\left(s_t, a_t\right)
$$

###### アルゴリズム

［入力］環境（状態集合 $\mathcal{S}$ と行動集合 $\mathcal{A}$ のみ既知．行動が入力されると，報酬と次状態を出力するブラックボックスなモデル），方策モデル $\pi(a \mid s ; \hat{Q}, t)$ ，割引率 $\gamma$ ，学習率 $\alpha_t$ ，終了条件（最大時間ステップ数 など）
［出力］最適行動価値の推定値 $\hat{Q}: \mathcal{S} \times \mathcal{A} \rightarrow \mathbb{R}$
1．初期化
- 推定値 $\hat{Q}: \mathcal{S} \times \mathcal{A} \rightarrow \mathbb{R}$ を任意に初期化．
- 時間ステップ $t$ を初期化 ：$t=0$ 。
- 初期状態 $s_0$ を環境から観測．

2．環境との相互作用
- $\pi_t\left(a \mid s_t ; \hat{Q}\right)$ に従い行動 $a_t$ を選択，$a_t$ を環境に入力．
- 環境から報酬 $r_t$ と次状態 $s_{t+1}$ を観測.

3．学習
－TD 誤差 $\delta$ を計算．

$$
\delta_t:=r_t+\gamma \max _{a^{\prime} \in \mathcal{A}} \hat{Q}\left(s_{t+1}, a^{\prime}\right)-\hat{Q}\left(s_t, a_t\right)
$$

－$\hat{Q}$ を更新．

$$
\hat{Q}\left(s_t, a_t\right):=\hat{Q}\left(s_t, a_t\right)+\alpha_t \delta_t
$$


4．終了判定
もし終了条件を満たしているならば，終了.
それ以外は，$t:=t+1$ として，手順 2 ．から繰り返す.

#### SARSA
６．SARSAを説明してください。
行動価値関数に関する方策のベルマン作用素と近似ベルマン作用素を以下のように定義します。
$$
\Upsilon_\pi q(s, a) \triangleq g(s, a)+\gamma \sum_{s^{\prime} \in \mathcal{S}} \sum_{a^{\prime} \in \mathcal{A}} p_{\mathrm{T}}\left(s^{\prime} \mid s, a\right) \pi\left(a^{\prime} \mid s^{\prime}\right) q\left(s^{\prime}, a^{\prime}\right), \forall(s, a) \in \mathcal{S} \times \mathcal{A}
$$

$$
\begin{aligned}
& \hat{\Upsilon}\left(q ; h_T\right)(s, a)\triangleq \begin{cases}\frac{\sum_{t=0}^{T-1} \mathbb{I}_{\left\{s_t=s\right\}} \mathbb{I}_{\left\{a_t=a\right\}}\left(r_t+\gamma q\left(s_{t+1}, a_{t+1}\right)\right)}{\sum_{t=0}^{T-1} \mathbb{I}_{\left\{s_t=s\right\}} \mathbb{I}_{\left\{a_t=a\right\}}}\left(\sum_{t=0}^{T-1} \mathbb{I}_{\left\{s_t=s\right\}} \mathbb{I}_{\left\{a_t=a\right\}}>0\right) \\
q(s, a) & \text { (それ以外) }, \forall(s, a) \in \mathcal{S} \times \mathcal{A}\end{cases}
\end{aligned}
$$

SARSA法はTD法と同様に以下のように定義されます。TD誤差も定義しておきます。

$$
\hat{Q}\left(s_t, a_t\right):=\left(1-\alpha_t\right) \hat{Q}\left(s_t, a_t\right)+\alpha_t \hat{\Upsilon}\left(\hat{Q} ;\left\{s_t, a_t, r_t, s_{t+1}, a_{t+1}\right\}\right)\left(s_t, a_t\right)
$$

形を変えると以下のようになります。

$$
\hat{Q}\left(s_t, a_t\right):=\hat{Q}\left(s_t, a_t\right)+\alpha_t \delta_t^{(\text {sarsa })}
$$

$$
\delta_t^{(\mathrm{sarsa})} \triangleq r_t+\gamma \hat{Q}\left(s_{t+1}, a_{t+1}\right)-\hat{Q}\left(s_t, a_t\right)
$$

#### Actor-Critic

７．アクタークリティック法(TD誤差を用いたバージョン)を説明せよ。

アクターにソフトマックス方策モデルを用い、クリティックでTD法を用いて価値関数を計算します。各タイムステップで計算されたTD誤差を用いて価値関数の更新と、方策の更新を行います。